## NBA Scheduling Problem

In [ ]:
!pip install ortools
!pip install geopy

In [ ]:
import numpy as np
import pandas as pd
# !pip install ortools
nba_teams = [
    "Atlanta Hawks",
    "Boston Celtics",
    "Brooklyn Nets",
    "Charlotte Hornets",
    "Chicago Bulls",
    "Cleveland Cavaliers",
    "Dallas Mavericks",
    "Denver Nuggets",
    "Detroit Pistons",
    "Golden State Warriors",
    "Houston Rockets",
    "Indiana Pacers",
    "Los Angeles Clippers",
    "Los Angeles Lakers",
    "Memphis Grizzlies",
    "Miami Heat",
    "Milwaukee Bucks",
    "Minnesota Timberwolves",
    "New Orleans Pelicans",
    "New York Knicks",
    "Oklahoma City Thunder",
    "Orlando Magic",
    "Philadelphia 76ers",
    "Phoenix Suns",
    "Portland Trail Blazers",
    "Sacramento Kings",
    "San Antonio Spurs",
    "Toronto Raptors",
    "Utah Jazz",
    "Washington Wizards"
]

num_teams = len(nba_teams)




```
Data Needed:
- List of all team names
- Distances between any two stadiums
- For each team:
  - Matchups they need to play throuhgout the year
  - Their divisions/conference
- Travel cost/fuel for each distance

teams and locations: https://github.com/carissaallen/NBA-Database/blob/master/CSV_Files/Teams.csv
```



In [ ]:
# Load Location Data
# !git clone https://github.com/carissaallen/NBA-Database.git
!git clone https://github.com/AdelHeddadji/nba_scheduling.git

In [ ]:
# Calculate distance function
from geopy.distance import geodesic
import pandas as pd

# Load the NBA team cities CSV file
nba_cities_df = pd.read_csv('nba_scheduling/nba_team_locations.csv')

def calculate_distance(team_index_1, team_index_2):
    """
    Calculate the geodesic distance in miles between two NBA team cities.

    Parameters:
        team1 (str): Name of the first NBA team.
        team2 (str): Name of the second NBA team.

    Returns:
        float: Distance in miles between the two cities.
    """

    team1 = nba_teams[team_index_1]
    team2 = nba_teams[team_index_2]



    # Find the locations of the two teams
    team1_data = nba_cities_df[nba_cities_df['Team'] == team1]
    team2_data = nba_cities_df[nba_cities_df['Team'] == team2]

    # Ensure both teams are found in the data
    if team1_data.empty or team2_data.empty:
        raise ValueError("One or both team names are not valid.")

    # Extract latitude and longitude
    team1_coords = (team1_data.iloc[0]['Latitude'], team1_data.iloc[0]['Longitude'])
    team2_coords = (team2_data.iloc[0]['Latitude'], team2_data.iloc[0]['Longitude'])

    # Calculate the distance using geopy
    distance = geodesic(team1_coords, team2_coords).miles
    return int(distance)


In [ ]:
from ortools.sat.python import cp_model

# Define constants
NUM_TEAMS = 30
NUM_GAMES = 58
NUM_GAME_DAYS = 136
HOME_AWAY_RATIO = NUM_GAMES // 2

distance_matrix = [
    [calculate_distance(team1, team2) for team2 in range(NUM_TEAMS)]
    for team1 in range(NUM_TEAMS)
]

flattened_distances = [distance_matrix[i][j] for i in range(NUM_TEAMS) for j in range(NUM_TEAMS)]

# Initialize the model
model = cp_model.CpModel()

# Variables
# schedule[day, team1, team2] = 1 if team1 plays against team2 on that day
schedule = {}
for day in range(NUM_GAME_DAYS):
    for team1 in range(NUM_TEAMS):
        for team2 in range(NUM_TEAMS):
            if team1 != team2:
                schedule[day, team1, team2] = model.NewBoolVar(f'schedule_{day}_{team1}_{team2}')

# locations[day, team] = location of team on day (an integer representing a team's ID)
locations = {}
for day in range(NUM_GAME_DAYS):
    for team in range(NUM_TEAMS):
        locations[day, team] = model.NewIntVar(0, NUM_TEAMS - 1, f'location_{day}_{team}')

# Constraints

# 1. Each team plays exactly NUM_GAMES games in total
for team in range(NUM_TEAMS):
    total_games_played = sum(
        schedule[day, opp, team] + schedule[day, team, opp]
        for opp in range(NUM_TEAMS) if opp != team
        for day in range(NUM_GAME_DAYS)
    )
    model.Add(total_games_played == NUM_GAMES)

# 2. Each team plays HOME_AWAY_RATIO home games
for team in range(NUM_TEAMS):
    home_games = sum(
        schedule[day, team, opp]
        for day in range(NUM_GAME_DAYS)
        for opp in range(NUM_TEAMS) if opp != team
    )
    model.Add(home_games == HOME_AWAY_RATIO)

# 3. At most one game per day for each team
for day in range(NUM_GAME_DAYS):
    for team in range(NUM_TEAMS):
        daily_games = sum(
            schedule[day, team, opp] + schedule[day, opp, team]
            for opp in range(NUM_TEAMS) if opp != team
        )
        model.Add(daily_games <= 1)

# 4. Initial locations: each team starts at their own stadium
for team in range(NUM_TEAMS):
    model.Add(locations[0, team] == team)

# 5. Update team locations based on previous day's game
for day in range(1, NUM_GAME_DAYS):
    for team in range(NUM_TEAMS):
        # Calculate played_home and played_away for previous day
        played_home = sum(schedule[day - 1, team, opp] for opp in range(NUM_TEAMS) if opp != team)
        played_away = sum(schedule[day - 1, opp, team] for opp in range(NUM_TEAMS) if opp != team)
        daily_games = played_home + played_away

        # Define booleans for scenarios
        home_day_bv = model.NewBoolVar(f'home_day_bv_{day}_{team}')
        away_day_bv = model.NewBoolVar(f'away_day_bv_{day}_{team}')
        no_game_day_bv = model.NewBoolVar(f'no_game_day_bv_{day}_{team}')

        # If they played at home, played_home == 1, played_away == 0
        model.Add(home_day_bv == played_home)

        # If they played away, played_away == 1, played_home == 0
        model.Add(away_day_bv == played_away)

        # If they did not play any game, daily_games = 0 => no_game_day_bv = 1 - daily_games = 1
        model.Add(no_game_day_bv == 1 - daily_games)

        # Ensure exactly one scenario is true (this is implied by construction, but let's be explicit)
        model.Add(home_day_bv + away_day_bv + no_game_day_bv == 1)

        # If home_day_bv is true, location = team's own stadium
        model.Add(locations[day, team] == team).OnlyEnforceIf(home_day_bv)

        # If no_game_day_bv is true, location stays the same as previous day
        model.Add(locations[day, team] == locations[day - 1, team]).OnlyEnforceIf(no_game_day_bv)

        # If away_day_bv is true, location is the stadium of the opponent they visited
        # Since only one game can be played, this sum returns the single opponent if any
        away_location_expr = sum(opp * schedule[day - 1, opp, team] for opp in range(NUM_TEAMS) if opp != team)
        model.Add(locations[day, team] == away_location_expr).OnlyEnforceIf(away_day_bv)

# 6. Minimize back-to-back games and travel distance
back_to_back = []
travel_distance = []
max_distance = int(max(max(row) for row in distance_matrix))

for day in range(NUM_GAME_DAYS - 1):
    for team in range(NUM_TEAMS):
        # Back-to-back games
        btb_var = model.NewBoolVar(f'btb_{day}_{team}')
        games_today = sum(
            schedule[day, team, opp] + schedule[day, opp, team]
            for opp in range(NUM_TEAMS) if opp != team
        )
        games_next_day = sum(
            schedule[day + 1, team, opp] + schedule[day + 1, opp, team]
            for opp in range(NUM_TEAMS) if opp != team
        )
        model.Add(btb_var >= games_today + games_next_day - 1)
        model.Add(btb_var <= games_today)
        model.Add(btb_var <= games_next_day)
        back_to_back.append(btb_var)

        # Travel distance
        travel_var = model.NewIntVar(0, max_distance, f'travel_{day}_{team}')
        travel_distance.append(travel_var)

        combined_index = model.NewIntVar(0, NUM_TEAMS * NUM_TEAMS - 1, f'combined_index_{day}_{team}')
        model.Add(combined_index == locations[day, team] * NUM_TEAMS + locations[day + 1, team])
        model.AddElement(
            combined_index,
            flattened_distances,
            travel_var
        )

# 7. Ensure every team plays every other team at least 2 times
for team1 in range(NUM_TEAMS):
    for team2 in range(NUM_TEAMS):
        if team1 != team2:
            games_between_teams = sum(
                schedule[day, team1, team2] + schedule[day, team2, team1]
                for day in range(NUM_GAME_DAYS)
            )
            model.Add(games_between_teams == 2)

# Objective
model.Minimize(sum(back_to_back) + sum(travel_distance) * (1/3000))

# Solve
solver = cp_model.CpSolver()
solver.parameters.cp_model_presolve = True
solver.parameters.linearization_level = 2
solver.parameters.cp_model_probing_level = 2
solver.parameters.num_search_workers = 16
solver.parameters.max_time_in_seconds = 60 * 300
status = solver.Solve(model)

# Output the solution
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Solution found!")
    if status == cp_model.OPTIMAL:
        print("OPTIMAL solution found!")
    else:
        print("FEASIBLE solution found!")
    for day in range(NUM_GAME_DAYS):
        print(f"Day {day + 1}:")
        for team1 in range(NUM_TEAMS):
            for team2 in range(NUM_TEAMS):
                if team1 != team2:
                    if solver.Value(schedule[day, team1, team2]):
                        print(f"  {nba_teams[team2]} @ {nba_teams[team1]}")
    print("ALL-STAR BREAK BEGINS (No games for the next week)")
else:
    print("No feasible solution found.")


In [ ]:
# Calculate and print the number of back-to-back games using back_to_back variables
total_back_to_back_games_1 = sum(solver.Value(btb_var) for btb_var in back_to_back)
print(f"Total back-to-back games (Days 1-128): {total_back_to_back_games_1}")

# Calculate and print the total traveled distance using travel_distance variables
total_travel_distance_1 = sum(solver.Value(travel_var) for travel_var in travel_distance)
print(f"Total travel distance (Days 1-128): {total_travel_distance_1}")

In [ ]:
from ortools.sat.python import cp_model

# Define constants
NUM_TEAMS = 30
NUM_GAMES = 24
NUM_GAME_DAYS = 54
HOME_AWAY_RATIO = NUM_GAMES // 2

distance_matrix = [
    [calculate_distance(team1, team2) for team2 in range(NUM_TEAMS)]
    for team1 in range(NUM_TEAMS)
]

flattened_distances = [distance_matrix[i][j] for i in range(NUM_TEAMS) for j in range(NUM_TEAMS)]

# Initialize the model
model = cp_model.CpModel()

# Variables
# schedule[day, team1, team2] = 1 if team1 plays against team2 on that day
schedule = {}
for day in range(NUM_GAME_DAYS):
    for team1 in range(NUM_TEAMS):
        for team2 in range(NUM_TEAMS):
            if team1 != team2:
                schedule[day, team1, team2] = model.NewBoolVar(f'schedule_{day}_{team1}_{team2}')

# locations[day, team] = location of team on day (an integer representing a team's ID)
locations = {}
for day in range(NUM_GAME_DAYS):
    for team in range(NUM_TEAMS):
        locations[day, team] = model.NewIntVar(0, NUM_TEAMS - 1, f'location_{day}_{team}')

# Constraints

# 1. Each team plays exactly NUM_GAMES games in total
for team in range(NUM_TEAMS):
    total_games_played = sum(
        schedule[day, opp, team] + schedule[day, team, opp]
        for opp in range(NUM_TEAMS) if opp != team
        for day in range(NUM_GAME_DAYS)
    )
    model.Add(total_games_played == NUM_GAMES)

# 2. Each team plays HOME_AWAY_RATIO home games
for team in range(NUM_TEAMS):
    home_games = sum(
        schedule[day, team, opp]
        for day in range(NUM_GAME_DAYS)
        for opp in range(NUM_TEAMS) if opp != team
    )
    model.Add(home_games == HOME_AWAY_RATIO)

# 3. At most one game per day for each team
for day in range(NUM_GAME_DAYS):
    for team in range(NUM_TEAMS):
        daily_games = sum(
            schedule[day, team, opp] + schedule[day, opp, team]
            for opp in range(NUM_TEAMS) if opp != team
        )
        model.Add(daily_games <= 1)

# 4. Initial locations: each team starts at their own stadium
for team in range(NUM_TEAMS):
    model.Add(locations[0, team] == team)

# 5. Update team locations based on previous day's game
for day in range(1, NUM_GAME_DAYS):
    for team in range(NUM_TEAMS):
        # Calculate played_home and played_away for previous day
        played_home = sum(schedule[day - 1, team, opp] for opp in range(NUM_TEAMS) if opp != team)
        played_away = sum(schedule[day - 1, opp, team] for opp in range(NUM_TEAMS) if opp != team)
        daily_games = played_home + played_away

        # Define booleans for scenarios
        home_day_bv = model.NewBoolVar(f'home_day_bv_{day}_{team}')
        away_day_bv = model.NewBoolVar(f'away_day_bv_{day}_{team}')
        no_game_day_bv = model.NewBoolVar(f'no_game_day_bv_{day}_{team}')

        # If they played at home, played_home == 1, played_away == 0
        model.Add(home_day_bv == played_home)

        # If they played away, played_away == 1, played_home == 0
        model.Add(away_day_bv == played_away)

        # If they did not play any game, daily_games = 0 => no_game_day_bv = 1 - daily_games = 1
        model.Add(no_game_day_bv == 1 - daily_games)

        # Ensure exactly one scenario is true (this is implied by construction, but let's be explicit)
        model.Add(home_day_bv + away_day_bv + no_game_day_bv == 1)

        # If home_day_bv is true, location = team's own stadium
        model.Add(locations[day, team] == team).OnlyEnforceIf(home_day_bv)

        # If no_game_day_bv is true, location stays the same as previous day
        model.Add(locations[day, team] == locations[day - 1, team]).OnlyEnforceIf(no_game_day_bv)

        # If away_day_bv is true, location is the stadium of the opponent they visited
        # Since only one game can be played, this sum returns the single opponent if any
        away_location_expr = sum(opp * schedule[day - 1, opp, team] for opp in range(NUM_TEAMS) if opp != team)
        model.Add(locations[day, team] == away_location_expr).OnlyEnforceIf(away_day_bv)

# 6. Minimize back-to-back games and travel distance
back_to_back = []
travel_distance = []
max_distance = int(max(max(row) for row in distance_matrix))

for day in range(NUM_GAME_DAYS - 1):
    for team in range(NUM_TEAMS):
        # Back-to-back games
        btb_var = model.NewBoolVar(f'btb_{day}_{team}')
        games_today = sum(
            schedule[day, team, opp] + schedule[day, opp, team]
            for opp in range(NUM_TEAMS) if opp != team
        )
        games_next_day = sum(
            schedule[day + 1, team, opp] + schedule[day + 1, opp, team]
            for opp in range(NUM_TEAMS) if opp != team
        )
        model.Add(btb_var >= games_today + games_next_day - 1)
        model.Add(btb_var <= games_today)
        model.Add(btb_var <= games_next_day)
        back_to_back.append(btb_var)

        # Travel distance
        travel_var = model.NewIntVar(0, max_distance, f'travel_{day}_{team}')
        travel_distance.append(travel_var)

        combined_index = model.NewIntVar(0, NUM_TEAMS * NUM_TEAMS - 1, f'combined_index_{day}_{team}')
        model.Add(combined_index == locations[day, team] * NUM_TEAMS + locations[day + 1, team])
        model.AddElement(
            combined_index,
            flattened_distances,
            travel_var
        )

# 7. Ensure every team plays every other team at least 2 times
for team1 in range(NUM_TEAMS):
    for team2 in range(NUM_TEAMS):
        if team1 != team2:
            games_between_teams = sum(
                schedule[day, team1, team2] + schedule[day, team2, team1]
                for day in range(NUM_GAME_DAYS)
            )
            model.Add(games_between_teams <= 1)

# Objective
model.Minimize(sum(back_to_back) + sum(travel_distance) * (1/3000))

# Solve
solver = cp_model.CpSolver()
solver.parameters.cp_model_presolve = True
solver.parameters.linearization_level = 2
solver.parameters.cp_model_probing_level = 2
solver.parameters.num_search_workers = 20
solver.parameters.max_time_in_seconds = 60 * 300
status = solver.Solve(model)

# Output the solution
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Solution found!")
    if status == cp_model.OPTIMAL:
        print("OPTIMAL solution found!")
    else:
        print("FEASIBLE solution found!")
    for day in range(NUM_GAME_DAYS):
        print("ALL-STAR BREAK ENDS (No games for the past week)")
        print(f"Day {day + 129}:")
        for team1 in range(NUM_TEAMS):
            for team2 in range(NUM_TEAMS):
                if team1 != team2:
                    if solver.Value(schedule[day, team1, team2]):
                        print(f"  {nba_teams[team2]} @ {nba_teams[team1]}")
else:
    print("No feasible solution found.")


In [ ]:
# Calculate and print the number of back-to-back games using back_to_back variables
total_back_to_back_games_2 = sum(solver.Value(btb_var) for btb_var in back_to_back)
print(f"Total back-to-back games (Days 129-190): {total_back_to_back_games_2}")

# Calculate and print the total traveled distance using travel_distance variables
total_travel_distance_2 = sum(solver.Value(travel_var) for travel_var in travel_distance)
print(f"Total travel distance (Days 129-190): {total_travel_distance_2}")

In [ ]:
print(f"Total back-to-back games: {total_back_to_back_games_1 + total_back_to_back_games_2}")
print(f"Total travel distance: {total_travel_distance_1 + total_travel_distance_2}")
print(f"Average back-to-back games per team: {(total_back_to_back_games_1 + total_back_to_back_games_2) / 30}")
print(f"Average travel distance per team: {(total_travel_distance_1 + total_travel_distance_2) / 30}")